<a href="https://colab.research.google.com/github/Manish-k723/100-days-of-DL/blob/main/GanWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import csv

In [55]:
def process_trades(trades):
    open_trades = {}
    paired_trades = []
    pnl_total = 0
    try:
      for trade in trades:
          time, symbol, side, price, quantity = trade
          price = float(price)
          quantity = int(quantity)
          if side == 'B':
              if symbol not in open_trades:
                  open_trades[symbol] = {'quantity': 0, 'price': 0, 'time': time}
              open_trades[symbol]['quantity'] += quantity
              open_trades[symbol]['price'] = price

          elif side == 'S':
              if symbol not in open_trades:
                  continue
              remaining_quantity = quantity
              while remaining_quantity > 0:
                  open_trade = open_trades[symbol]
                  paired_quantity = min(remaining_quantity, open_trade['quantity'])
                  pnl = paired_quantity * (price - open_trade['price'])
                  pnl_total += pnl

                  paired_trades.append({
                      'open_time': open_trade['time'],
                      'close_time': time,
                      'symbol': symbol,
                      'quantity': paired_quantity,
                      'pnl': pnl,
                      'open_side': 'B',
                      'close_side': 'S',
                      'open_price': open_trade['price'],
                      'close_price': price
                  })

                  remaining_quantity -= paired_quantity
                  open_trade['quantity'] -= paired_quantity

                  if open_trade['quantity'] == 0:
                      del open_trades[symbol]

      return paired_trades, pnl_total
    except:
      del open_trades[symbol]

In [56]:
def print_paired_trades(paired_trades):
    header = "OPEN_TIME, CLOSE_TIME, SYMBOL, QUANTITY, PNL, OPEN_SIDE, CLOSE_SIDE, OPEN_PRICE, CLOSE_PRICE"
    print(header)

    for trade in sorted(paired_trades, key=lambda x: x['close_time']):
        print(f"{trade['open_time']},{trade['close_time']},{trade['symbol']},{trade['quantity']},{trade['pnl']:.2f},"
              f"{trade['open_side']},{trade['close_side']},{trade['open_price']:.2f},{trade['close_price']:.2f}")

In [57]:
with open('/content/demo_trades.csv', 'r') as file:
        reader = csv.reader(file)
        header = next(reader)
        trades = [row for row in reader]

paired_trades, pnl_total = process_trades(trades)

print_paired_trades(paired_trades)
print(f"Total PNL: {pnl_total:.2f}")

KeyError: ignored